In [63]:
# Settings

In [1]:
#%%
import argparse
import os
import torchvision.transforms as transforms
import pathlib
import torch
from PIL import Image
import seaborn as sns
import pandas as pd
IMAGE_EXTENSIONS = {'bmp', 'jpg', 'jpeg', 'pgm', 'png', 'ppm',
                    'tif', 'tiff', 'webp'}
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from time import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from fid_score import fid_score
from prdc import compute_prdc
from is_score import inception_score
from EmbedNet import EmbedNetwork
import prd_score as prd
try:
    from tqdm import tqdm
except ImportError:
    # If tqdm is not available, provide a mock version of it
    def tqdm(x):
        return x

In [2]:
class Args():
    def __init__(self):
        # reference data의 폴더명
        self.ref_dir = 'datas/REAL-6080-1'
        # evaluation data의 폴더명 (리스트로 받는다.)
        self.eval_dirs = ['datas/DCGAN-6080-1',
                         'datas/DCGAN-6080-2',
                         'datas/DCGAN-6080-3',
                         'datas/PGGAN-6080-2']
        self.embeddingModel = 'inceptionV3'
        self.isscore = True
        self.fid=True
        self.pr=True
        self.iprdc=True
        self.save_numpy=False
        self.finetuned = True # imagenet pretrained == False or finetuned network == True
        self.verbose = True
        
        # input your finetuned_path (str)
        self.finetuned_path = 'finetuned/alllayerfinetuned-inceptionv3-220219.pth'

args = Args()

In [3]:
def load_dataloader(directory):
    nc = 3
    files = path_from_dir(directory=directory)
    imgs = load_images_from_dir(files, nc)
    return imgs


# inceptionV3을 통해서 임베딩을 한다.
def load_or_generate_embedding(directory, model):
    if args.finetuned:
        embeddingfilename = f'{directory}/{args.embeddingModel}_finetuned_embeddings.npy'
    else:
        embeddingfilename = f'{directory}/{args.embeddingModel}_pretrained_embeddings.npy'
    
    files = path_from_dir(directory=directory)
    if os.path.isfile(embeddingfilename):
        if args.verbose:
                print(f'Loading {args.embeddingModel} embeddings for ' + directory)
        embeddings = np.load(embeddingfilename)
    else:
        nc = 3
        # 디렉토리로부터 이미지를 갖고온다.
        imgs = load_images_from_dir(files, nc)
        embeddings = model.modelEmbedding(imgs, args)
    if args.save_numpy:
        np.save(embeddingfilename, embeddings)
    return embeddings, files


class ImagePathDataset(torch.utils.data.Dataset):
    def __init__(self, files, nc, transforms=None):
        self.files = files
        self.transforms = transforms
        self.nc = nc

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        path = self.files[i]
        if self.nc == 3:
            img = Image.open(path).convert('RGB')
        else:
            img = Image.open(path).convert('L')
        if self.transforms is not None:
            img = self.transforms(img)
        return img

def path_from_dir(directory, types=('png', 'jpg', 'bmp', 'gif')):
    print(directory)
    directory = pathlib.Path(directory)
    files = sorted([file for ext in IMAGE_EXTENSIONS
                    for file in directory.glob('*.{}'.format(ext))])
    return files

# 디렉토리로부터 이미지를 갖고온다.
def load_images_from_dir(files, nc):
    transform = []
    transform.extend([transforms.ToTensor()])
    transform.extend([transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])])
    dataset = ImagePathDataset(files, nc, transforms=transforms.Compose(transform))
    dataloader = torch.utils.data.DataLoader(dataset,
                                             batch_size=32,
                                             shuffle=True,
                                             drop_last=False,
                                             num_workers=8)

    return dataloader



device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')

In [4]:

# ref 폴더 경로와 eval 폴더 경로의 절대 경로를 얻는다.
ref_dir = os.path.abspath(args.ref_dir)
eval_dirs = [os.path.abspath(directory) for directory in args.eval_dirs]
eval_embeddingsLists = []

feature_dim = 1000
nearest_k = 5
nearest_kList = [3, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
pathdata = []

network = EmbedNetwork(net_name=args.embeddingModel, device=device, finetuned=args.finetuned, finetuned_path = args.finetuned_path)

print(ref_dir)
if args.verbose:
    print('computing inception embeddings for ' + ref_dir)
real_embeddings, _realpath = load_or_generate_embedding(
    ref_dir, network)
_reallabel= ref_dir.split('/')[-1]


prd_data = []
for directory in eval_dirs:
    if args.verbose:
        print('computing inception embeddings for ' + directory)
    eval_embeddings, _fakepath = load_or_generate_embedding(
        directory, network)
    _evallabel = directory.split('/')[-1]
    eval_embeddingsLists.append(eval_embeddings)

    if args.verbose:
        print('finish embedding')
path = np.concatenate((_realpath, _fakepath), axis=0)

print('original pretrained weight state')
print('model is ', args.embeddingModel)

result_df = dict()
result_kpdf = dict()
result_krdf = dict()
result_kddf= dict()
result_kcdf= dict()


index_metrics = ['is', 'fid', 'precision', 'recall', 'improved precision', 'improved recall', 'density', 'coverage']
index_k = [str(k) for k in nearest_kList]

for eval_embeddings, directory_name in zip(eval_embeddingsLists, eval_dirs):
    directory = directory_name.split('/')[-1]
    print('Evaluating ', directory)
    print(f'setting {args.embeddingModel}-finetuned:{str(args.finetuned)}')
    result_df[directory] = []
    result_kpdf[directory] = []
    result_krdf[directory] = []
    result_kddf[directory] = []
    result_kcdf[directory] = []
    print()

    if args.isscore:
        imgs = load_dataloader(directory_name)
        print("Calculating Inception Score...")
        print('*'*20)
        print(directory_name)
        is_result = inception_score(imgs=imgs, device=device, batch_size=32,
            resize=True, splits=10, finetuned=args.finetuned, finetuned_path = args.finetuned_path)
        print(is_result[0])
        result_df[directory].append(is_result[0])
        print('*'*20)

    if args.fid:
        print('computing FID')
        print('*'*20)
        fids = fid_score(real_embeddings, eval_embeddings)
        print('FID is ', fids)
        result_df[directory].append(fids[0])
        print('*'*20)
    if args.pr:
        # default setting
        beta = 8
        num_clusters = 20
        num_angles = 1001
        num_runs = 10
        print('computing precision recall distribution')
        precision, recall = prd.compute_prd_from_embedding(
        eval_data=eval_embeddings,
        ref_data=real_embeddings,
        num_clusters=num_clusters,
        num_angles=num_angles,
        num_runs=num_runs)

        if args.verbose:
            print('plotting results')
        f_beta_data = prd.prd_to_max_f_beta_pair(precision, recall, beta=beta)
        print('F_8(Recall)   F_1/8(Precision)     model')
        print('%.3f %.3f     ' % (f_beta_data[0], f_beta_data[1]))
        result_df[directory].append(f_beta_data[1])
        result_df[directory].append(f_beta_data[0])
        print('*'*20)


    if args.iprdc:
        print('computing Improved Precision Recall and Density Coverage')
        prdc = compute_prdc(real_features=real_embeddings,
                    fake_features=eval_embeddings,
                    nearest_k=nearest_k)
        print('Nearest K value is ', nearest_k, ' and Directory PRDC: ', directory, prdc)
        result_df[directory].append(prdc['precision'])
        result_df[directory].append(prdc['recall'])
        result_df[directory].append(prdc['density'])
        result_df[directory].append(prdc['coverage'])
        print('*'*20)
        print()
        print('k computing')
        for k in nearest_kList:
            prdc = compute_prdc(real_features=real_embeddings,
                    fake_features=eval_embeddings,
                    nearest_k=k)
            print(k)
            result_kpdf[directory].append(prdc['precision'])
            result_krdf[directory].append(prdc['recall'])
            result_kddf[directory].append(prdc['density'])
            result_kcdf[directory].append(prdc['coverage'])

        print('finish')
        print('*'*20)

metrics = pd.DataFrame(result_df, columns=args.eval_dirs, index=index_metrics)
imprvp = pd.DataFrame(result_kpdf, columns=args.eval_dirs, index=index_k)
imprvr = pd.DataFrame(result_krdf, columns=args.eval_dirs, index=index_k)
density = pd.DataFrame(result_kddf, columns=args.eval_dirs, index=index_k)
coverage = pd.DataFrame(result_kcdf, columns=args.eval_dirs, index=index_k)

result_save_dir  = f'setting-{args.embeddingModel}-finetuned:{str(args.finetuned)}'
if not os.path.isdir(result_save_dir):
    os.mkdir(result_save_dir)

print('finish')
# %%

{'finetuned_path': 'finetuned/alllayerfinetuned-inceptionv3-220219.pth'}
load finetuned
load medical finetuned weights (imagenet + medical)
/home/softjin/datas/REAL-6080-1
computing inception embeddings for /home/softjin/datas/REAL-6080-1
/home/softjin/datas/REAL-6080-1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:18<00:00, 10.44it/s]


shape (6080, 2048)
computing inception embeddings for /home/softjin/datas/DCGAN-6080-1
/home/softjin/datas/DCGAN-6080-1
Loading inceptionV3 embeddings for /home/softjin/datas/DCGAN-6080-1
finish embedding
computing inception embeddings for /home/softjin/datas/DCGAN-6080-2
/home/softjin/datas/DCGAN-6080-2
Loading inceptionV3 embeddings for /home/softjin/datas/DCGAN-6080-2
finish embedding
computing inception embeddings for /home/softjin/datas/DCGAN-6080-3
/home/softjin/datas/DCGAN-6080-3
Loading inceptionV3 embeddings for /home/softjin/datas/DCGAN-6080-3
finish embedding
computing inception embeddings for /home/softjin/datas/PGGAN-6080-2
/home/softjin/datas/PGGAN-6080-2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:18<00:00, 10.52it/s]


shape (6080, 2048)
finish embedding
original pretrained weight state
model is  inceptionV3
Evaluating  DCGAN-6080-1
setting inceptionV3-finetuned:True

/home/softjin/datas/DCGAN-6080-1
Calculating Inception Score...
********************
/home/softjin/datas/DCGAN-6080-1
load model weight
load finetuned


...

1.1164542194842633
********************
computing FID
********************
FID is  (120.74016943740378, 120.74017498838553)
********************
computing precision recall distribution
plotting results
F_8(Recall)   F_1/8(Precision)     model
0.843 0.897     
********************
computing Improved Precision Recall and Density Coverage
Num real: 6080 Num fake: 6080
Nearest K value is  5  and Directory PRDC:  DCGAN-6080-2 {'precision': 0.4, 'recall': 0.49046052631578946, 'density': 0.16384868421052634, 'coverage': 0.22615131578947367}
********************

k computing
Num real: 6080 Num fake: 6080
3
Num real: 6080 Num fake: 6080
5
Num real: 6080 Num fake: 6080
6
Num real: 6080 Num fake: 6080
7
Num real: 6080 Num fake: 6080
8
Num real: 6080 Num fake: 6080
9
Num real: 6080 Num fake: 6080
10
Num real: 6080 Num fake: 6080
20
Num real: 6080 Num fake: 6080
30
Num real: 6080 Num fake: 6080
40
Num real: 6080 Num fake: 6080
50
Num real: 6080 Num fake: 6080
60
Num real: 6080 Num fake: 6080
70
Num

/home/softjin/evaluation-merics/is_score.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


1.1328336981837501
********************
computing FID
********************
FID is  (121.4766057668329, 121.47660174825532)
********************
computing precision recall distribution
plotting results
F_8(Recall)   F_1/8(Precision)     model
0.831 0.906     
********************
computing Improved Precision Recall and Density Coverage
Num real: 6080 Num fake: 6080
Nearest K value is  5  and Directory PRDC:  DCGAN-6080-3 {'precision': 0.3988486842105263, 'recall': 0.49161184210526315, 'density': 0.16151315789473686, 'coverage': 0.22450657894736842}
********************

k computing
Num real: 6080 Num fake: 6080
3
Num real: 6080 Num fake: 6080
5
Num real: 6080 Num fake: 6080
6
Num real: 6080 Num fake: 6080
7
Num real: 6080 Num fake: 6080
8
Num real: 6080 Num fake: 6080
9
Num real: 6080 Num fake: 6080
10
Num real: 6080 Num fake: 6080
20
Num real: 6080 Num fake: 6080
30
Num real: 6080 Num fake: 6080
40
Num real: 6080 Num fake: 6080
50
Num real: 6080 Num fake: 6080
60
Num real: 6080 Num fak

/home/softjin/evaluation-merics/is_score.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


1.184455713135803
********************
computing FID
********************
FID is  (6.404118974457788, 6.404118904196685)
********************
computing precision recall distribution
plotting results
F_8(Recall)   F_1/8(Precision)     model
0.990 0.993     
********************
computing Improved Precision Recall and Density Coverage
Num real: 6080 Num fake: 6080
Nearest K value is  5  and Directory PRDC:  PGGAN-6080-2 {'precision': 0.8876644736842105, 'recall': 0.9504934210526316, 'density': 0.7188486842105264, 'coverage': 0.8529605263157894}
********************

k computing
Num real: 6080 Num fake: 6080
3
Num real: 6080 Num fake: 6080
5
Num real: 6080 Num fake: 6080
6
Num real: 6080 Num fake: 6080
7
Num real: 6080 Num fake: 6080
8
Num real: 6080 Num fake: 6080
9
Num real: 6080 Num fake: 6080
10
Num real: 6080 Num fake: 6080
20
Num real: 6080 Num fake: 6080
30
Num real: 6080 Num fake: 6080
40
Num real: 6080 Num fake: 6080
50
Num real: 6080 Num fake: 6080
60
Num real: 6080 Num fake: 60

In [5]:
save_path = result_save_dir

metrics =pd.DataFrame(result_df, index=index_metrics)
metrics.to_csv(save_path + '/' + 'metrics.csv', sep=',', na_rep='NaN')
metrics

,DCGAN-6080-1,DCGAN-6080-2,DCGAN-6080-3,PGGAN-6080-2
is,1.113600,1.116454,1.132834,1.184456
fid,120.031759,120.740169,121.476606,6.404119
precision,0.895971,0.897452,0.905757,0.992712
recall,0.847953,0.842971,0.831245,0.990111
improved precision,0.400164,0.400000,0.398849,0.887664
improved recall,0.494408,0.490461,0.491612,0.950493
density,0.158914,0.163849,0.161513,0.718849
coverage,0.219408,0.226151,0.224507,0.852961


In [6]:
imprvp = pd.DataFrame(result_kpdf, index=index_k)
imprvp.to_csv(save_path + '/' + 'imprvp_k.csv', sep=',', na_rep='NaN')
imprvp

,DCGAN-6080-1,DCGAN-6080-2,DCGAN-6080-3,PGGAN-6080-2
3,0.199671,0.202632,0.204112,0.706579
5,0.400164,0.400000,0.398849,0.887664
6,0.470066,0.466776,0.474342,0.920066
7,0.540461,0.531414,0.537171,0.941776
8,0.595559,0.587993,0.580921,0.956250
9,0.639474,0.629934,0.622039,0.965132
10,0.671546,0.665132,0.661020,0.973355
20,0.855592,0.857895,0.853618,0.994079
30,0.920230,0.918750,0.915461,0.997697
40,0.948849,0.951316,0.945395,0.998684


In [7]:
imprvr = pd.DataFrame(result_krdf, index=index_k)
imprvr.to_csv(save_path + '/' + 'imprvr_k.csv', sep=',', na_rep='NaN')
imprvr

,DCGAN-6080-1,DCGAN-6080-2,DCGAN-6080-3,PGGAN-6080-2
3,0.354770,0.334211,0.336349,0.877796
5,0.494408,0.490461,0.491612,0.950493
6,0.554441,0.549178,0.546053,0.962829
7,0.604441,0.601151,0.590461,0.971053
8,0.648684,0.641118,0.636513,0.976645
9,0.683059,0.677138,0.673520,0.980263
10,0.713158,0.713158,0.702138,0.983882
20,0.869737,0.878947,0.878618,0.996546
30,0.930099,0.933224,0.936184,0.998849
40,0.955099,0.961349,0.961184,0.999671


In [8]:
density = pd.DataFrame(result_kddf, index=index_k)
density.to_csv(save_path + '/' + 'density_k.csv', sep=',', na_rep='NaN')
density

,DCGAN-6080-1,DCGAN-6080-2,DCGAN-6080-3,PGGAN-6080-2
3,0.093366,0.097423,0.096382,0.550219
5,0.158914,0.163849,0.161513,0.718849
6,0.182812,0.184759,0.185883,0.758882
7,0.206955,0.204981,0.207965,0.786372
8,0.226295,0.223931,0.224013,0.809108
9,0.240424,0.240534,0.241667,0.829094
10,0.255428,0.253240,0.256480,0.841036
20,0.355831,0.353396,0.359704,0.903553
30,0.413964,0.409419,0.419079,0.920428
40,0.448199,0.440785,0.452488,0.930933


In [9]:
coverage = pd.DataFrame(result_kcdf, index=index_k)
coverage.to_csv(save_path + '/' + 'coverage_k.csv', sep=',', na_rep='NaN')
coverage

,DCGAN-6080-1,DCGAN-6080-2,DCGAN-6080-3,PGGAN-6080-2
3,0.087500,0.095888,0.089803,0.588651
5,0.219408,0.226151,0.224507,0.852961
6,0.284211,0.284539,0.287993,0.905592
7,0.343421,0.344408,0.350493,0.936349
8,0.398849,0.401480,0.400000,0.960691
9,0.449671,0.453947,0.452632,0.973355
10,0.503454,0.499342,0.498684,0.983059
20,0.745724,0.741118,0.743092,0.998684
30,0.847862,0.841283,0.835362,0.999836
40,0.898026,0.891118,0.887500,1.000000
